# Can we find the 2^n modulus mapping for all ints in the lattice?

We know for k(mod 8) there is a 1:1 correspondence between the modulus of a given integer and the 1/8 of the lattice it will be found int.

Can we extend this to higher moduli AND dtermine the mapping pattern no matter how large the number?

Suspect that (mod 8) being 3 bits is special in this regard due to the Collatz Conjecture ending in 3 //2 operations.


In [81]:
import sys, io
import math
import numpy as np
import pandas as pd
import random
import re
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint
from itertools import product
from sympy.ntheory import factorint
from typing import List, Optional, Tuple

In [82]:
"""
Code from previous notebooks,

TODO: Remove unneed functions not needed for mrTup representation

"""

def T(n):
    """ Compute next value in simplified Collatz sequence.
    """
    if n & 1 == 0:
        return n//2
    else:
        return (3*n + 1)//2
#
def L_T(n):
    """ Compute binary label-string for a given Collatz number
    """
    if n == 1:
        return "1"

    S = ""
    while n != 1:
        if n & 1 == 0:
            n = n//2
            S = S + "1"
        else:
            n = (3*n + 1)//2
            S = S + "0"
    return S
#
def Ay_L(L):
    """ Generate A matrix and y vector from label string
    """
    rank = len(L) + 2    
    A = np.zeros((rank,rank))
    y = np.zeros((rank))
    for row in range(rank-3):
        if L[row-2] == "0":
            a_val = -1.0
            y_val = 0.0
        else:
            a_val = -3.0
            y_val = 1.0
        A[row][row] = a_val
        A[row][row+1] = 2.0
        y[row] = y_val
    #
    # Last 3 rows are always the same
    row = rank - 3
    A[row][row] = -1
    A[row][row+1] = 2
    y[row] = 0
    row = rank - 2
    A[row][row] = -3
    A[row][row+1] = 2
    y[row] = 1
    row = rank - 1
    A[row][row] = 1
    A[row][row-2] = -1
    y[row] = 0
    
    return A, y
#
def solve_Ay_L(L):
    """ Solve for the x vector given the label-string
    """
    A, y = Ay_L(L)
    return A, np.linalg.solve(A, y), y
#
def x0_L(L):
    """ Get the x[0] value given a label-string
    """
    A, x, y = solve_Ay_L(L)
    return round(x[0])  # clean up mantisa garbage
#

def countZeros(label):
    zero_count = 0
    for bit in label:
        if bit == "0":
            zero_count += 1
    return zero_count
#

def Z(L):
    """ Indexes of zeros in label string
    """
    for i in range(len(L)):
        if L[i] == "0":
            yield i
#
def a_b_c_L(L):
    """ Get the (power-of-two, power-of-three, zero-sum-accumulator) tuple for a node given its label
    """
    a = len(L)
    b = 0
    for bit in L:
        if bit == "0":
            b += 1
    ZZ = [(j,i) for j, i in enumerate(Z(L))]
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in ZZ)
    S = [zz[1] for zz in ZZ]
    return (a,b,c,S)
#
def val_a_b_c(a_b_c):
    """ Get the value for a node given the tuple (power-of-two, power-of-three, zero-sum-accumulator)
    """
    a, b, c = a_b_c
    f = Fraction( ((2**a) - c), (3**b) )
    return (f.numerator, f.denominator)
#
def val_a_b_c_L(a_b_c_L):
    a_b_c = a_b_c_L[0:3]
    return val_a_b_c(a_b_c)
#
def val_L(L):
    """ Get the value for a node given the label string
    """
    return val_a_b_c_L(a_b_c_L(L))
#
def collatzPath(collatzNumber):
    if collatzNumber < 1:
        return None
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#

def collatzPath2(n_d_tup, truncate_at=100):
    """
    Collatz Path for rationals with loop detection and "too long" cut-off
    """
    chain = [n_d_tup]
    path = []
    governor = truncate_at
    while n_d_tup != (1, 1):
        if (n_d_tup[0] & 1) == 0:
            n_d_tup = (n_d_tup[0]//2, n_d_tup[1])
            path.append("1")
        else:
            f = Fraction((3 * n_d_tup[0] + n_d_tup[1])//2, n_d_tup[1])
            n_d_tup = (f.numerator, f.denominator)
            path.append("0")
        if n_d_tup in chain:
            path.insert(0,"↺")
            break
        
        governor -= 1
        if governor == 0:
            path.insert(0,"∀")
            break
        #
        chain.append(n_d_tup)
    #
    return ("".join(path), chain)
#

    
#
#  Mixed Radix form and functions
#

N_ = ((0,0), [])

def mr_TupItemValue(a_b, a_0):
    a,b = a_b
    val = (2**a)*(3**(a_0 + b))
    # print(f"val {a_b}*(3**{a_0}) = {val}")
    return val
#
def mrTupValue(mr_tup):
    # Multiplying the numerator by 3 ** the generation keeps us in integer land
    a_0 = mr_tup[0][0]
    total = mr_TupItemValue(mr_tup[0], a_0)
    for a_b in mr_tup[1]:
        total -= mr_TupItemValue(a_b, a_0)
    frac = Fraction(total, 3**a_0)
    return (frac.numerator, frac.denominator)
#

def F_0(mr_tup):
    return ( (mr_tup[0][0]+1, mr_tup[0][1]-1), mr_tup[1] + [(mr_tup[0][0], -(len(mr_tup[1])+1))] )
#

def F_1(mr_tup):
    return ((mr_tup[0][0]+1, mr_tup[0][1]), mr_tup[1])
#
def mrTupFromPath(label):
    mr_tup = N_
    for bit in label:
        if bit == "1":
            mr_tup = F_1(mr_tup)
        else:
            mr_tup = F_0(mr_tup)
    return mr_tup
#
def mrTupToPath(T):
    """
    Convert mrTup to node label
    The T[1] list encodes the positions of the zeros in the numerator power of two values
    """
    S = ["1"] * T[0][0]
    for j in range(len(T[1])):
        S[(T[1][j][0])] = "0" 
    return "".join(S)
#

def mrTupFromValue(n):
    label = collatzPath(n)
    return mrTupFromPath(label)
#

def strip_01(label):
    while len(label) > 2 and label[-2:] == "01":
        label = label[0:-2]
    return label
#

LABEL_RX = re.compile('^(?P<prefix>.*?)((?P<inttag>111)(?P<tail>((01)*)))$')

def split_int_label(s):
    """
    Splits label into 3 parts and returns prefix and suffix if matches integer-candidate pattern
    """
    match = LABEL_RX.search(s)
    if match:
        return (match.group('prefix'), match.group('tail'))
    else:
        return None  # or raise an error if preferred
#

def generationLabels(a):
    if a == 0:
        return ""
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        yield label
#
def generationTups(a):
    for label in generationLabels(a):
        mrTup = mrTupFromPath(label)
        yield (label, mrTup, mrTupValue(mrTup))
    
def generationIntCandidateLabels(a):
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        head_tail = split_int_label(label)
        if head_tail:
            # return int candidate with stripped tail
            yield head_tail[0] + "111"
#

def mrIntTupsForGeneration(aa):
    for label in generationIntCandidateLabels(aa):
        mrTup = mrTupFromPath(label)
        yield (label, mrTup, mrTupValue(mrTup))
#
def generationGenNums(a):
    vals = []
    bb = 3**(a)
    for infoTup in generationTups(a):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] * (bb//val_tup[1]))
    vals.sort(reverse=True)
    for idx, val in enumerate(vals):
        if idx < len(vals) - 1:
            delta = val - vals[idx+1]
        else:
            delta = None
        print(f'{val}\t{delta}')
#        
def generationPairGenNums(a):
    vals = []
    bb = 3**(a+1)
    for infoTup in generationTups(a):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] * (bb//val_tup[1]))
    for infoTup in generationTups(a+1):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] *  (bb//val_tup[1]))
    vals.sort(reverse=True)
    for idx, val in enumerate(vals):
        if idx < len(vals) - 1:
            delta = val - vals[idx+1]
        else:
            delta = None
        print(f'{val}\t{delta}')
#
def mr2Nplus_1(T):
    B = len(T[1])  
    L = [(0, -1)]

    # Keep initial zeros
    idx = 0
    for idx, val in enumerate(T[1]):
        if T[1][idx][0] != idx:
            break
        L.append( (T[1][idx][0] + 1, T[1][idx][1]-1) )
    # Remove the first tuple where (a, -a) is true
    match = False
    for i in range(idx, B, 1):
        if (not match) and (T[1][i][0] == -T[1][i][1]):
            match = True
        else:
            L.append( (T[1][i][0]+1, T[1][i][1]) )
    if not match:
        return None
    return ( (T[0][0] + 1, T[0][1]), L)
#

'''
Do not think this works yet:
def mr2Nplus_1_inv(T):
    """
    Find the inverse tuple of mr2Nplus_1

    We know the zero was removed when L item index matched numerator and denominator
    """
    B = len(T[1])  
    L = []

    # Keep initial zeros after poping (0, -1) off the front
    idx = 0
    for idx, val in enumerate(T[1]):
        if idx == 0:
            if val == (0, -1):
                continue
            else:
                # This tuple was not generated by mr2Nplus_1
                return None
        else:
            L.append( (T[1][idx][0] - 1, T[1][idx][1] + 1) )
    inserted = False
    for i in range(idx, B, 1):
        if (not inserted) and i == (len(L)+1) and (T[0][i][0] > i):
            # Next zero in 2n+1 L list is larger than i ... insert the (i, -i) term
            L.append( (i, -i) )    
            inserted == True
        L.append( (T[1][i][0]-1, T[1][i][1]) )
        
    return ( (T[0][0] - 1, T[0][1]), L)
#
'''

def lattice2N_plus1_pairs(a):
    """
    For a given depth in the tree, generate all 2n+1 pairs in the tree
    """
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        label = strip_01(label)
        val = mrTupValue(mrTupFromPath(label))
        f = Fraction(2 * val[0] + 1, val[1])
        val_ = (f.numerator, f.denominator)
        label_, chain_ = collatzPath2(val_)
        if (len(label_) == 0):
            d = len(label)
        elif label_[0] in ["↺", "∀"]:
            d = 100 + len(label)
        else:
            d = distance(label, label_)
        yield (len(label), d, (val, label), (val_, label_))
#

def mrTupToLaTex(T):
    a, b = T[0]
    s = "\\frac{2^{%d}}{3^{%d}}"%(a, -b)
    L = T[1]
    if len(L) > 0:
        s = s + " - ( "
        plus = "  "
        for c_d in L:
            c, d = c_d
            t = "\\frac{2^{%d}}{3^{%d}}"%(c, -d)
            s = s + plus + t
            plus = " + "
        s = s + " )"
    return "$ " + s + " $"
#

In [83]:
def checkTupEasy(label):
    if len(label) < 4:
        label = label + "0101"
    T = mrTupFromPath(label)
    TT = mr2Nplus_1(T)
    if TT is not None:
        return True
    else:
        return False
#

In [84]:
PARTS_RX_tail = re.compile('^(?P<head>[01]*?)((?P<inttag>(111)?)(?P<tail>((01)*)))$')
PARTS_RX_head = re.compile('^(?P<prefix>0*)(?P<ones>1*)(?P<thezero>0?)(?P<remainder>[01]*)$')
RULE1_RX = re.compile('^(?P<prefix>0*)(?P<ones>1+)(?P<thezero>0)(?P<remainder>[01]*)((?P<inttag>111)(?P<tail>((01)*)))$')

In [85]:
def mrTupLabelParts(label):
    match_tail = PARTS_RX_tail.search(label)
    if match_tail:
        match_head = PARTS_RX_head.search(match_tail.group('head'))
        if match_head:
            P = (match_head.group('prefix'), match_head.group('ones'), match_head.group('thezero'), match_head.group('remainder'), match_tail.group('inttag'), match_tail.group('tail'))
    else:
        P = ("","","",label, "", "")
    return P
#

In [86]:
def genEasy2n_plus_1(a):
    """
    Generate all "easy" 2n+1 mrTups
    """
    if a <= 5:
        yield None
    else:
        for label in generationLabels(a):
            T = mrTupFromPath(label)
            val = mrTupValue(T)
            if val[1] == 1:
                did_match = False
                match = RULE1_RX.search(label)
                if match:
                    if a <= 12:
                        print((match.group('prefix'), match.group('ones'), match.group('thezero'), match.group('remainder'), match.group('inttag'), match.group('tail')))
                    did_match = True
                    T_ = mr2Nplus_1(T)
                    if T_ is None:
                        print("EASY NOT EASY:")
                        print((val[0], label))
                    else:
                        val_ = mrTupValue(T_)
                        if  val_[1] != 1 or val_[0] != (2*val[0] + 1):
                            print("EASY NOT EASY -- check fail:")
                            print((val[0], label))
                yield((label, val, did_match))
#

In [87]:
def mr2Nplus_1(T):
    B = len(T[1])  
    L = [(0, -1)]

    # Keep initial zeros
    idx = 0
    for idx, val in enumerate(T[1]):
        if T[1][idx][0] != idx:
            break
        L.append( (T[1][idx][0] + 1, T[1][idx][1]-1) )
    # Remove the first tuple where (a, -a) is true
    match = False
    for i in range(idx, B, 1):
        if (not match) and (T[1][i][0] == -T[1][i][1]):
            match = True
        else:
            L.append( (T[1][i][0]+1, T[1][i][1]) )
    if not match:
        return None
    return ( (T[0][0] + 1, T[0][1]), L)
#


In [88]:
def splitOutInitialZeros(L):
    L_a = []
    L_z = []
    initial = True
    for i in range(len(L)):
        if initial and L[i][0] == i:
            L_a.append(L[i])
        else:
            initial = False
            L_z.append(L[i])
    return L_a, L_z
#


In [89]:
T_27 = mrTupFromValue(27)
print(splitOutInitialZeros(T_27[1]))

([(0, -1), (1, -2)], [(3, -3), (4, -4), (5, -5), (6, -6), (7, -7), (9, -8), (11, -9), (12, -10), (14, -11), (15, -12), (16, -13), (18, -14), (19, -15), (20, -16), (21, -17), (23, -18), (26, -19), (27, -20), (28, -21), (30, -22), (31, -23), (33, -24), (34, -25), (35, -26), (36, -27), (37, -28), (38, -29), (41, -30), (42, -31), (43, -32), (44, -33), (48, -34), (50, -35), (52, -36), (56, -37), (59, -38), (60, -39), (61, -40), (66, -41)])


In [90]:

def mrTup2NIdentityElement(T):
    """ 
    returns the index of the L list item. if any, that allows
    a 2n operation to be transformed into a 2n+1 operation by
    the removal of a single element.

    Brute force for now
    """
    val = mrTupValue(T)
    f = Fraction(2*val[0] + val[1], val[1])
    val_goal = (f.numerator, f.denominator)

    a = T[0][0]
    b = T[0][1]
    L = T[1]

    L_head, L_tail = splitOutInitialZeros(L)

    L_head_ = [(0, -1)]
    for c_d in L_head:
        L_head_.append((c_d[0] + 1, c_d[1] -1))
    
    for i in range(len(L_tail)):
        # which one of these, when we ditch it gives us our goal value?
        L_tail_ = [(c_d[0] + 1, c_d[1]) for idx, c_d in enumerate(L_tail) if idx != i]
        T_ = ((a+1, b), L_head_ + L_tail_)
        # print(T_)
        val_ = mrTupValue(T_)
        if (val_ == val_goal):
            # i seems to always be zero
            return (i, i + len(L_head_))
    return None
#

In [91]:
n = 16
aa = 10
D = {}
for a in range(2, aa):
    D[a] = {}
for i in range(2, 2**n + 2, 1):
    label = collatzPath(i)
    for a in range(2, aa):
        if len(label) >= a:
            prefix = label[0:a]
            mod = i % (2**a)
            if (prefix) not in D[a]:
                D[a][prefix] = {}
            if (mod) not in D[a][prefix]:
                D[a][prefix][mod] = 0
            D[a][prefix][mod] += 1
#

In [92]:
for a in D:
    for prefix in D[a]:
        if len(D[a][prefix]) > 1:
            print((a, prefix, D[a][prefix].keys()))

# Prefixes of length(a) continue to be 1:1 with numbers $\pmod{2^a}$

Can we find the pattern?



In [93]:
def printD_map(a):
    for prefix in D[a]:
        mod_nums = list(D[a][prefix].keys())
        if len(mod_nums) > 1:
            print("UNEXPECTED")
        else:
            mod_num = mod_nums[0]
            mod_num_str = bin(mod_num)[2:].zfill(a)
            print(f"{prefix}\t{mod_num_str}")

In [94]:
printD_map(2)

00	11
11	00
01	01
10	10


In [95]:
printD_map(3)

001	011
011	101
100	110
000	111
111	000
010	001
101	010
110	100


# a LSB to MSB expansion !

# $\pmod{2^{2}} \mapsto \pmod{2^{3}} $

- 00 -> 11
  - 00**0** -> **1**11
  - 00**1** -> **0**11 
- 01 -> 01
  - 01**0** -> **0**01
  - 01**1** -> **1**01
- 10 -> 10
  - 10**0** -> **1**10
  - 10**1** -> **0**10
- 11 -> 00
  - 11**0** -> **1**00
  - 11**1** -> **0**00

# $\pmod{2^{3}} \mapsto \pmod{2^{4}} $

- 000 -> 111
  - 000**0** -> **1**111
  - 000**1** -> **0**111
- 001 -> 011
  - 001**0** -> **1**011
  - 001**1** -> **0**011
- 010 -> 001
  - 010**0** -> **1**001
  - 010**1** -> **0**001
- 011 -> 101
  - 011**0** -> **1**101
  - 011**1** -> **0**101
- 100 -> 110
  - 100**0** -> **1**110
  - 100**1** -> **0**110
- 101 -> 010
  - 101**0** -> **0**010
  - 101**1** -> **1**010
- 110 -> 100
  - 110**0** -> **1**100
  - 110**1** -> **0**100
- 111 -> 000
  - 111**0** -> **1**000
  - 111**1** -> **0**000

# $\pmod{2^{4}} \mapsto \pmod{2^{5}} $

- 0000 -> 1111
  - 0000**0** -> **1**1111
  - 0000**1** -> **0**1111
- 0001 -> 0111
  - 0001**0** -> **0**0111
  - 0001**1** -> **1**0111
- 0010 -> 1011
  - 0010**0** -> **1**1011
  - 0010**1** -> **0**1011
- 0011 -> 0011
  - 0011**0** -> **1**0011
  - 0011**1** -> **0**0011
- 0100 -> 1001
  - 0100**0** -> **0**1001
  - 0100**1** -> **1**1001
- 0101 -> 0001
  - 0101**0** -> **0**0001
  - 0101**1** -> **1**0001
- 0110 -> 1101
  - 0110**0** -> **1**1101
  - 0110**1** -> **0**1101
- 0111 -> 0101
  - 0111**0** -> **0**0101
  - 0111**1** -> **1**0101
- 1000 -> 1110
  - 1000**0** -> **1**1110
  - 1000**1** -> **0**1110
- 1001 -> 0110
  - 1001**0** -> **1**0110
  - 1001**1** -> **0**0110
- 1010 -> 0010
  - 1010**0** -> **1**0010
  - 1010**1** -> **0**0010
- 1011 -> 1010
  - 1011**0** -> **1**1010
  - 1011**1** -> **0**1010
- 1100 -> 1100
  - 1100**0** -> **1**1100
  - 1100**1** -> **0**1100
- 1101 -> 0100
  - 1101**0** -> **0**0100
  - 1101**1** -> **1**0100
- 1110 -> 1000
  - 1110**0** -> **1**1000
  - 1110**1** -> **0**1000
- 1111 -> 0000
  - 1111**0** -> **1**0000
  - 1111**1** -> **0**0000

In [96]:
def formatLSB_MSB(a):
    for i in range(2**a):
        prefix = bin(i)[2:].zfill(a)
        mod_vals = list(D[a][prefix].keys())
        mod_val = mod_vals[0]
        mod_val_str = bin(mod_val)[2:].zfill(a)
        print(f"- {prefix} -> {mod_val_str}")
        a_ = a + 1
        for bit in ["0", "1"]:
            prefix_ = prefix + bit
            mod_vals = list(D[a_][prefix_].keys())
            mod_val = mod_vals[0]
            mod_val_str = bin(mod_val)[2:].zfill(a_)
            print(f"  - {prefix}**{bit}** -> **{mod_val_str[0]}**{mod_val_str[1:]}")
#
formatLSB_MSB(3)

- 000 -> 111
  - 000**0** -> **1**111
  - 000**1** -> **0**111
- 001 -> 011
  - 001**0** -> **1**011
  - 001**1** -> **0**011
- 010 -> 001
  - 010**0** -> **1**001
  - 010**1** -> **0**001
- 011 -> 101
  - 011**0** -> **1**101
  - 011**1** -> **0**101
- 100 -> 110
  - 100**0** -> **1**110
  - 100**1** -> **0**110
- 101 -> 010
  - 101**0** -> **0**010
  - 101**1** -> **1**010
- 110 -> 100
  - 110**0** -> **1**100
  - 110**1** -> **0**100
- 111 -> 000
  - 111**0** -> **1**000
  - 111**1** -> **0**000


In [97]:
formatLSB_MSB(4)

- 0000 -> 1111
  - 0000**0** -> **1**1111
  - 0000**1** -> **0**1111
- 0001 -> 0111
  - 0001**0** -> **0**0111
  - 0001**1** -> **1**0111
- 0010 -> 1011
  - 0010**0** -> **1**1011
  - 0010**1** -> **0**1011
- 0011 -> 0011
  - 0011**0** -> **1**0011
  - 0011**1** -> **0**0011
- 0100 -> 1001
  - 0100**0** -> **0**1001
  - 0100**1** -> **1**1001
- 0101 -> 0001
  - 0101**0** -> **0**0001
  - 0101**1** -> **1**0001
- 0110 -> 1101
  - 0110**0** -> **1**1101
  - 0110**1** -> **0**1101
- 0111 -> 0101
  - 0111**0** -> **0**0101
  - 0111**1** -> **1**0101
- 1000 -> 1110
  - 1000**0** -> **1**1110
  - 1000**1** -> **0**1110
- 1001 -> 0110
  - 1001**0** -> **1**0110
  - 1001**1** -> **0**0110
- 1010 -> 0010
  - 1010**0** -> **1**0010
  - 1010**1** -> **0**0010
- 1011 -> 1010
  - 1011**0** -> **1**1010
  - 1011**1** -> **0**1010
- 1100 -> 1100
  - 1100**0** -> **1**1100
  - 1100**1** -> **0**1100
- 1101 -> 0100
  - 1101**0** -> **0**0100
  - 1101**1** -> **1**0100
- 1110 -> 1000
  - 1110**0** -> **

In [98]:
int("0010000010110010",2)

8370

In [99]:
collatzPath(12)

'1100111'

In [100]:
pattern_01 = ["0010", "00100000", "0010000010110010"]
for p in pattern_01:
    print(int(p, 2))
for p in pattern_01:
    print(int(p[::-1], 2))


2
32
8370
4
4
19716


In [101]:
def extract_01_bit_patterns(a):
    bits = []
    for i in range(2**a):
        prefix = bin(i)[2:].zfill(a)
        mod_vals = list(D[a][prefix].keys())
        mod_val = mod_vals[0]
        mod_val_str = bin(mod_val)[2:].zfill(a)
        a_ = a + 1
        prefix_ = prefix + "0"
        mod_vals = list(D[a_][prefix_].keys())
        mod_val = mod_vals[0]
        mod_val_str = bin(mod_val)[2:].zfill(a_)
        if mod_val_str[0] == "1":
            bits.append("1")
        else:
            bits.append("0")
    bit_string = "".join(bits)
    return bit_string[::-1]
            
#    

In [102]:
def show_01_bit_patterns(a):
    for i in range(2**a):
        prefix = bin(i)[2:].zfill(a)
        T = mrTupFromPath(prefix)
        val = mrTupValue(T)
        mod_vals = list(D[a][prefix].keys())
        mod_val = mod_vals[0]
        mod_val_str = bin(mod_val)[2:].zfill(a)
        a_ = a + 1
        
        prefix_0 = prefix + "0"
        mod_vals = list(D[a_][prefix_0].keys())
        modulus_0 = mod_vals[0]
        modulus_0_str = bin(modulus_0)[2:].zfill(a_)
        T_0 = mrTupFromPath(prefix_0)
        val_0 = mrTupValue(T_0)

        prefix_1 = prefix + "1"
        mod_vals = list(D[a_][prefix_1].keys())
        modulus_1 = mod_vals[0]
        modulus_1_str = bin(modulus_1)[2:].zfill(a_)        
        T_1 = mrTupFromPath(prefix_1)
        val_1 = mrTupValue(T_1)
        
        print(f"{prefix}({val}) is_mod {mod_val}({mod_val_str})")
        print(f"    {prefix_0}({val_0}) is_mod {modulus_0}({modulus_0_str})")
        print(f"    {prefix_1}({val_1}) is_mod {modulus_1}({modulus_1_str})")

            
#    

In [103]:
def computeNextPrefixBit(a, p2, label, mod):
    # We cannot choose two label bits, so we generate an exemplar
    example = p2 + mod 
    label = collatzPath(example) 
    return a+1, 2**(a+1), label[0:a]
    
def computePrefix(n):
    mod = n % 4
    label = ["11", "01", "10", "00"][mod]
    if n > 3:
        a = 3
        p2 = 2**(a)
        while  p2 < n:
            a, p2, label = computeNextPrefixBit(a, p2, label, n % p2)
    return label

In [104]:
computePrefix(347)

'00100010'

In [105]:
mrTupValue(mrTupFromPath('00100010'))

(-733, 729)

In [106]:
collatzPath(347)

'00100010111110000010100100010000101100010010000001100001110101011101100011110111'

In [107]:
len(computePrefix(347347347347347347347))

68

In [108]:
# '00100010111110000010100100010000101100010010000001100001110101011101100011110111'
# '00100010'
# '11111111111110000010100100010000101100010010000001100001110101011101100011110111'

In [109]:
mrTupValue(mrTupFromPath('11111111111110000010100100010000101100010010000001100001110101011101100011110111'))

(253952, 1)

In [110]:
mrTupValue(mrTupFromPath('0000010100100010000101100010010000001100001110101011101100011110111'))

(31, 1)

In [111]:
mrTupValue(mrTupFromPath(computePrefix(347347347347347347347)))

(136166636809545274283, 16677181699666569)

In [112]:
136166636809545274283/16677181699666569

8164.846990440099

In [113]:
len(collatzPath(347347347347347347347))

273

In [114]:
len(computePrefix(347347347347347347347347347347347347347347))

137

In [115]:
len(collatzPath(347347347347347347347347347347347347347347))

750

In [116]:
def predict_01_bit_patterns(a, correct_mapping=D):
    for label in generationLabels(a-1):
        label_modulus = list(D[a-1][label].keys())[0]  # There is only ever 1: TODO: create getter that fills in cache on demand
        label_modulus_str = bin(label_modulus)[2:].zfill(a-1) # [2:] to strip '0b' from binary string
        val = mrTupValue(mrTupFromPath(label))
        label_0 = label + "0"
        label_1 = label + "1"
        val_0 = mrTupValue(mrTupFromPath(label_0))
        val_1 = mrTupValue(mrTupFromPath(label_1))

        # Our modulus candidates are:
        mod_u = label_modulus
        mod_v = label_modulus + 2**(a-1)
        # print((mod_u, mod_v))
        
        # We need examples (+1 in exponential is so we don't get too short of a label for smaller numbers)
        example_u = 2**(a+1) + mod_u 
        example_v = 2**(a+1) + mod_v

        label_u = collatzPath(example_u)
        label_v = collatzPath(example_v)  # not really necessary since the a-th bit given by one has to the opposite of the other

        #print(f"{example_u} -> {label_u}[a]={label_u[a-1]}")
        #print(f"{example_v} -> {label_v}[a]={label_v[a-1]}")

        if label_u[a-1] == "0":
            modulus_0 = mod_u
            modulus_1 = mod_v
        else:
            modulus_0 = mod_v
            modulus_1 = mod_u

        modulus_0_str = bin(modulus_0)[2:].zfill(a)
        modulus_1_str = bin(modulus_1)[2:].zfill(a)
        
        if modulus_0 not in correct_mapping[a][label_0]:
            print("MODULUS Mismatch on 0")
        if modulus_1 not in correct_mapping[a][label_1]:
            print("MODULUS Mismatch on 1")
        print(f"{label}({val}) is_mod {label_modulus}({label_modulus_str})")
        print(f"    {label_0}({val_0}) is_mod {modulus_0}({modulus_0_str})")
        print(f"    {label_1}({val_1}) is_mod {modulus_1}({modulus_1_str})")


In [117]:
predict_01_bit_patterns(3)

11((4, 1)) is_mod 0(00)
    110((4, 3)) is_mod 4(100)
    111((8, 1)) is_mod 0(000)
10((2, 3)) is_mod 2(10)
    100((-2, 9)) is_mod 6(110)
    101((2, 1)) is_mod 2(010)
01((1, 1)) is_mod 1(01)
    010((1, 9)) is_mod 1(001)
    011((7, 3)) is_mod 5(101)
00((-1, 9)) is_mod 3(11)
    000((-11, 27)) is_mod 7(111)
    001((1, 3)) is_mod 3(011)


In [118]:
show_01_bit_patterns(2)

00((-1, 9)) is_mod 3(11)
    000((-11, 27)) is_mod 7(111)
    001((1, 3)) is_mod 3(011)
01((1, 1)) is_mod 1(01)
    010((1, 9)) is_mod 1(001)
    011((7, 3)) is_mod 5(101)
10((2, 3)) is_mod 2(10)
    100((-2, 9)) is_mod 6(110)
    101((2, 1)) is_mod 2(010)
11((4, 1)) is_mod 0(00)
    110((4, 3)) is_mod 4(100)
    111((8, 1)) is_mod 0(000)


In [119]:
predict_01_bit_patterns(3)

11((4, 1)) is_mod 0(00)
    110((4, 3)) is_mod 4(100)
    111((8, 1)) is_mod 0(000)
10((2, 3)) is_mod 2(10)
    100((-2, 9)) is_mod 6(110)
    101((2, 1)) is_mod 2(010)
01((1, 1)) is_mod 1(01)
    010((1, 9)) is_mod 1(001)
    011((7, 3)) is_mod 5(101)
00((-1, 9)) is_mod 3(11)
    000((-11, 27)) is_mod 7(111)
    001((1, 3)) is_mod 3(011)


In [120]:
show_01_bit_patterns(2)

00((-1, 9)) is_mod 3(11)
    000((-11, 27)) is_mod 7(111)
    001((1, 3)) is_mod 3(011)
01((1, 1)) is_mod 1(01)
    010((1, 9)) is_mod 1(001)
    011((7, 3)) is_mod 5(101)
10((2, 3)) is_mod 2(10)
    100((-2, 9)) is_mod 6(110)
    101((2, 1)) is_mod 2(010)
11((4, 1)) is_mod 0(00)
    110((4, 3)) is_mod 4(100)
    111((8, 1)) is_mod 0(000)


In [121]:
predict_01_bit_patterns(3)

11((4, 1)) is_mod 0(00)
    110((4, 3)) is_mod 4(100)
    111((8, 1)) is_mod 0(000)
10((2, 3)) is_mod 2(10)
    100((-2, 9)) is_mod 6(110)
    101((2, 1)) is_mod 2(010)
01((1, 1)) is_mod 1(01)
    010((1, 9)) is_mod 1(001)
    011((7, 3)) is_mod 5(101)
00((-1, 9)) is_mod 3(11)
    000((-11, 27)) is_mod 7(111)
    001((1, 3)) is_mod 3(011)


In [122]:
predict_01_bit_patterns(4)

111((8, 1)) is_mod 0(000)
    1110((8, 3)) is_mod 8(1000)
    1111((16, 1)) is_mod 0(0000)
110((4, 3)) is_mod 4(100)
    1100((-4, 9)) is_mod 12(1100)
    1101((4, 1)) is_mod 4(0100)
101((2, 1)) is_mod 2(010)
    1010((2, 9)) is_mod 2(0010)
    1011((14, 3)) is_mod 10(1010)
100((-2, 9)) is_mod 6(110)
    1000((-22, 27)) is_mod 14(1110)
    1001((2, 3)) is_mod 6(0110)
011((7, 3)) is_mod 5(101)
    0110((5, 9)) is_mod 13(1101)
    0111((5, 1)) is_mod 5(0101)
010((1, 9)) is_mod 1(001)
    0100((-13, 27)) is_mod 9(1001)
    0101((1, 1)) is_mod 1(0001)
001((1, 3)) is_mod 3(011)
    0010((-7, 27)) is_mod 11(1011)
    0011((11, 9)) is_mod 3(0011)
000((-11, 27)) is_mod 7(111)
    0000((-49, 81)) is_mod 15(1111)
    0001((-1, 9)) is_mod 7(0111)


In [123]:
mrTupValue(((3, -2), [(0, -1), (2, -2)]))

(1, 9)

In [124]:
mrTupValue(((4, -2), [(0, -1), (2, -2)])), mrTupValue(((4, -3), [(0, -1), (2, -2), (3, -3)])) 

((1, 1), (-13, 27))

In [125]:
for j in range(5, 50, 8):
    i = j - 1
    label_4 = collatzPath(i)
    label_5 = collatzPath(j)
    print(label_4)
    print(label_5)
    print("-----")

11
0111
-----
1100111
0110111
-----
110111
011111
-----
1100010110111
0110010110111
-----
110100010110111
011100010110111
-----
110010110111
011010110111
-----


In [126]:
# In general when we swap '011' for '110' WE ALWAYS GET 1 even from Rationals
for label_tail in generationLabels(8):
    label_4 = "110" + label_tail
    label_5 = "011" + label_tail
    val_4 = mrTupValue(mrTupFromPath(label_4))
    val_5 = mrTupValue(mrTupFromPath(label_5))
    val_diff = Fraction(*val_5) - Fraction(*val_4)
    if val_diff != 1:
        print((label_4, label_5))

$$
F_{tail}(F_0(F_1(F_1(N_{\emptyset})))) -1 = F_{tail}(F_1(F_1(F_0(N_{\emptyset})))) 
$$

$$
F_0(F_1(F_1(N_{\emptyset}))) -1 = F_1(F_1(F_0(N_{\emptyset})))
$$

$$
V(T_{011}) -1 = V(T_{110})
$$

$$ \frac{7}{3} - 1 = \frac{4}{3} $$


# $3n+2$ Neighborly label transform

$n' = 3n+2$ Can be accomplished WHEN the label starts with $101$ by changing the start to $111$

In [127]:
for label_tail in generationLabels(8):
    label_101 = "101" + label_tail + "111"
    label_111 = "111" + label_tail + "111"    
    val_101 = mrTupValue(mrTupFromPath(label_101))
    if val_101[1] == 1:
        val_111 = mrTupValue(mrTupFromPath(label_111))
        print((len(label_101), val_101[0], val_111[0], val_101[0] % 8, val_111[0] % 8, label_101, label_111))
#
for label_tail in generationLabels(9):
    label_101 = "101" + label_tail + "111"
    label_111 = "111" + label_tail + "111"    
    val_101 = mrTupValue(mrTupFromPath(label_101))
    if val_101[1] == 1:
        val_111 = mrTupValue(mrTupFromPath(label_111))
        print((len(label_101), val_101[0], val_111[0], val_101[0] % 8, val_111[0] % 8, label_101, label_111))



(14, 1706, 5120, 2, 0, '10111111110111', '11111111110111')
(14, 554, 1664, 2, 0, '10111110110111', '11111110110111')
(14, 1818, 5456, 2, 0, '10110111111111', '11110111111111')
(14, 602, 1808, 2, 0, '10110101111111', '11110101111111')
(14, 186, 560, 2, 0, '10110011110111', '11110011110111')
(14, 58, 176, 2, 0, '10110010110111', '11110010110111')
(14, 18, 56, 2, 0, '10100010110111', '11100010110111')
(15, 10922, 32768, 2, 0, '101111111111111', '111111111111111')
(15, 3626, 10880, 2, 0, '101111101111111', '111111101111111')
(15, 1130, 3392, 2, 0, '101111011110111', '111111011110111')
(15, 362, 1088, 2, 0, '101111010110111', '111111010110111')
(15, 1210, 3632, 2, 0, '101100111111111', '111100111111111')
(15, 122, 368, 2, 0, '101100011110111', '111100011110111')
(15, 402, 1208, 2, 0, '101000111111111', '111000111111111')


In [128]:
# Do 2(mod 8) numbers always start with 101?  YES
for i in range(10, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "101":
        print((i, label))

In [129]:
# Do 0(mod 8) numbers always start with 111?  YES
for i in range(16, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "111":
        print((i, label))

In [130]:
# Do 1(mod 8) numbers always start with 010?  YES
for i in range(17, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "010":
        print((i, label))

In [131]:
# Do 7(mod 8) numbers always start with 000?  Yes
for i in range(15, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "000":
        print((i, label))

In [132]:
# Do 4(mod 8) numbers always start with 110?  Yes
for i in range(12, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "110":
        print((i, label))

In [133]:
# Do 6(mod 8) numbers always start with 100?  Yes
for i in range(14, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "100":
        print((i, label))

In [134]:
# Do 3(mod 8) numbers always start with 001?  Yes
for i in range(11, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "001":
        print((i, label))

In [135]:
# Do 5(mod 8) numbers always start with 011?  Yes
for i in range(13, 2000, 8):
    label = collatzPath(i)
    if label[0:3] != "011":
        print((i, label))

In [136]:
# Since 7 (mod 8) is the slowest grower, can we always convert it to everything else? YES!!!
PREFIXES =  ["000", "001", "010", "011", "100", "101", "110", "111"]
status =[[0]*8, [0]*8]
for i in range(15, 10000, 8):
    label = collatzPath(i)
    for prefix in [1,2,3,4,5,6,7]:
        label_ = PREFIXES[prefix] + label
        val_ = mrTupValue(mrTupFromPath(label))
        if val_[1] == 1:
            status[1][prefix] += 1
        else:
            status[0][prefix] += 1
status


[[0, 0, 0, 0, 0, 0, 0, 0], [0, 1249, 1249, 1249, 1249, 1249, 1249, 1249]]

# 1:1 mappings through 3 bit prefix changes:
- 0 (mod 8):
- 1 (mod 8): 0, 4, 5
- 2 (mod 8): 0
- 3 (mod 8): 0, 2
- 4 (mod 8): 0, 4, 5, 6
- 5 (mod 8): 0, 6
- 6 (mod 8): 0, 4, 5
- 7 (mod 8): 0, 4, 5, 6
- 

In [153]:
# Since 7 (mod 8) is the slowest grower, can we always convert it to everything else? YES!!!
PREFIXES =  ["111", "010", "101", "001", "110", "011", "100", "000"]
MOD =       [  0,     1,     2,     3,     4,     5,     6,     7  ]
for j in range(8):
    k = MOD[j]
    status =[[0]*8, [0]*8]
    for i in range(k+16, 10000, 8):
        label = collatzPath(i)
        for prefix in range(8):
            label_ = PREFIXES[prefix] + label[3:]
            val_ = mrTupValue(mrTupFromPath(label_))
            if val_[1] == 1:
                status[1][prefix] += 1
            else:
                status[0][prefix] += 1
    print((k, PREFIXES[j], status))


(0, '111', [[0, 1109, 832, 1109, 832, 832, 1110, 1202], [1248, 139, 416, 139, 416, 416, 138, 46]])
(1, '010', [[0, 0, 1248, 1248, 0, 0, 1248, 1248], [1248, 1248, 0, 0, 1248, 1248, 0, 0]])
(2, '101', [[0, 1248, 0, 832, 1248, 1248, 1248, 1248], [1248, 0, 1248, 416, 0, 0, 0, 0]])
(3, '001', [[0, 1248, 0, 0, 1248, 1248, 1248, 1248], [1248, 0, 1248, 1248, 0, 0, 0, 0]])
(4, '110', [[0, 832, 1248, 1248, 0, 0, 832, 1110], [1248, 416, 0, 0, 1248, 1248, 416, 138]])
(5, '011', [[0, 832, 1248, 1248, 0, 0, 832, 1110], [1248, 416, 0, 0, 1248, 1248, 416, 138]])
(6, '100', [[0, 1248, 1248, 1248, 0, 0, 0, 832], [1248, 0, 0, 0, 1248, 1248, 1248, 416]])
(7, '000', [[0, 1248, 1248, 1248, 0, 0, 0, 0], [1248, 0, 0, 0, 1248, 1248, 1248, 1248]])


In [138]:
# Since 7 (mod 8) is the slowest grower, can we always convert it to everything else? YES!!!
PREFIXES =  ["111", "010", "101", "001", "110", "011", "100", "000"]
MOD =       [  0,     1,     2,     3,     4,     5,     6,     7  ]
for j in range(8):
    k = MOD[j]
    status =[[0]*8, [0]*8]
    for i in range(k+16, 10000, 8):
        label = collatzPath(i)
        for prefix in range(8):
            label_ = PREFIXES[prefix] + "1" + label[3:]
            val_ = mrTupValue(mrTupFromPath(label_))
            if val_[1] == 1:
                status[1][prefix] += 1
            else:
                status[0][prefix] += 1
    print((k, PREFIXES[j], status))


(0, '111', [[0, 1110, 832, 1109, 832, 832, 1109, 1202], [1248, 138, 416, 139, 416, 416, 139, 46]])
(1, '010', [[0, 1248, 0, 0, 1248, 1248, 1248, 1248], [1248, 0, 1248, 1248, 0, 0, 0, 0]])
(2, '101', [[0, 832, 1248, 1248, 0, 0, 832, 1109], [1248, 416, 0, 0, 1248, 1248, 416, 139]])
(3, '001', [[0, 1248, 1248, 1248, 0, 0, 0, 832], [1248, 0, 0, 0, 1248, 1248, 1248, 416]])
(4, '110', [[0, 1248, 0, 832, 1248, 1248, 1248, 1248], [1248, 0, 1248, 416, 0, 0, 0, 0]])
(5, '011', [[0, 1248, 0, 832, 1248, 1248, 1248, 1248], [1248, 0, 1248, 416, 0, 0, 0, 0]])
(6, '100', [[0, 1248, 0, 1248, 1248, 1248, 1248, 1248], [1248, 0, 1248, 0, 0, 0, 0, 0]])
(7, '000', [[0, 1248, 0, 1248, 1248, 1248, 1248, 1248], [1248, 0, 1248, 0, 0, 0, 0, 0]])


# 1:1 mappings through (3 bit + "1") prefix changes:
- 0 (mod 8):
- 1 (mod 8): 2, 3
- 2 (mod 8): 0, 4, 5
- 3 (mod 8): 0, 4, 5, 6
- 4 (mod 8): 0, 2
- 5 (mod 8): 0, 2
- 6 (mod 8): 0, 2
- 7 (mod 8): 0, 2
- 

In [139]:
# Since 7 (mod 8) is the slowest grower, can we always convert it to everything else? YES!!!
PREFIXES =  ["111", "010", "101", "001", "110", "011", "100", "000"]
MOD =       [  0,     1,     2,     3,     4,     5,     6,     7  ]
for j in range(8):
    k = MOD[j]
    status =[[0]*8, [0]*8]
    for i in range(k+16, 10000, 8):
        label = collatzPath(i)
        for prefix in range(8):
            label_ = "1" + PREFIXES[prefix] + label[3:]
            val_ = mrTupValue(mrTupFromPath(label_))
            if val_[1] == 1:
                status[1][prefix] += 1
            else:
                status[0][prefix] += 1
    print((k, PREFIXES[j], status))


(0, '111', [[0, 1109, 832, 1109, 832, 832, 1110, 1202], [1248, 139, 416, 139, 416, 416, 138, 46]])
(1, '010', [[0, 0, 1248, 1248, 0, 0, 1248, 1248], [1248, 1248, 0, 0, 1248, 1248, 0, 0]])
(2, '101', [[0, 1248, 0, 832, 1248, 1248, 1248, 1248], [1248, 0, 1248, 416, 0, 0, 0, 0]])
(3, '001', [[0, 1248, 0, 0, 1248, 1248, 1248, 1248], [1248, 0, 1248, 1248, 0, 0, 0, 0]])
(4, '110', [[0, 832, 1248, 1248, 0, 0, 832, 1110], [1248, 416, 0, 0, 1248, 1248, 416, 138]])
(5, '011', [[0, 832, 1248, 1248, 0, 0, 832, 1110], [1248, 416, 0, 0, 1248, 1248, 416, 138]])
(6, '100', [[0, 1248, 1248, 1248, 0, 0, 0, 832], [1248, 0, 0, 0, 1248, 1248, 1248, 416]])
(7, '000', [[0, 1248, 1248, 1248, 0, 0, 0, 0], [1248, 0, 0, 0, 1248, 1248, 1248, 1248]])


# 1:1 mappings through ("1" + 3 bit) prefix changes:
- 0 (mod 8):
- 1 (mod 8): 0, 4, 5
- 2 (mod 8): 0, 2
- 3 (mod 8): 0, 4, 5, 6
- 4 (mod 8): 0, 5, 6
- 5 (mod 8): 0, 4
- 6 (mod 8): 0, 4, 5
- 7 (mod 8): 0, 4, 5, 6


In [140]:
for label_tail in generationLabels(8):
    label_101 = "101" + label_tail + "111"
    label_111 = "0111" + label_tail + "111"    
    val_101 = mrTupValue(mrTupFromPath(label_101))
    if val_101[1] == 1:
        val_111 = mrTupValue(mrTupFromPath(label_111))
        print((len(label_101), val_101[0], val_111[0], val_101[0] % 8, val_111[0] % 8, label_101, label_111))
#


(14, 1706, 3413, 2, 5, '10111111110111', '011111111110111')
(14, 554, 1109, 2, 5, '10111110110111', '011111110110111')
(14, 1818, 3637, 2, 5, '10110111111111', '011110111111111')
(14, 602, 1205, 2, 5, '10110101111111', '011110101111111')
(14, 186, 373, 2, 5, '10110011110111', '011110011110111')
(14, 58, 117, 2, 5, '10110010110111', '011110010110111')
(14, 18, 37, 2, 5, '10100010110111', '011100010110111')


In [141]:
for label_tail in generationLabels(9):
    label_101 = "101" + label_tail + "111"
    label_111 = "0111" + label_tail + "111"    
    val_101 = mrTupValue(mrTupFromPath(label_101))
    if val_101[1] == 1:
        val_111 = mrTupValue(mrTupFromPath(label_111))
        print((len(label_101), val_101[0], val_111[0], val_101[0] % 8, val_111[0] % 8, label_101, label_111))


(15, 10922, 21845, 2, 5, '101111111111111', '0111111111111111')
(15, 3626, 7253, 2, 5, '101111101111111', '0111111101111111')
(15, 1130, 2261, 2, 5, '101111011110111', '0111111011110111')
(15, 362, 725, 2, 5, '101111010110111', '0111111010110111')
(15, 1210, 2421, 2, 5, '101100111111111', '0111100111111111')
(15, 122, 245, 2, 5, '101100011110111', '0111100011110111')
(15, 402, 805, 2, 5, '101000111111111', '0111000111111111')


# The values of the 3bit mrTups

In [142]:
def changePrefix(label, n, new_prefix):
    return new_prefix + label[n:]
#

In [143]:
for n_mod_8 in MOD:
    prefix = PREFIXES[n_mod_8]
    val = mrTupValue(mrTupFromPath(prefix))
    print(f"{n_mod_8}\t{prefix}\t{val}")

0	111	(8, 1)
1	010	(1, 9)
2	101	(2, 1)
3	001	(1, 3)
4	110	(4, 3)
5	011	(7, 3)
6	100	(-2, 9)
7	000	(-11, 27)


# Formulas for all 1:1 3 bit substitutions

## 1 (mod 8): 0, 4, 5
- 0:  1/9 -> 8   ; (17 -> 160, 25 -> 232) ; $9 \cdot T_{010} + 7 \mapsto T_{111}$
- 4:  1/9 -> 4/3 ; (17 -> 52,  25 -> 76) ; $3 \cdot T_{010} + 1 \mapsto T_{110}$
- 5:  1/9 -> 7/3 ; (17 -> 53,  25 -> 77) ; $3 \cdot T_{010} + 2 \mapsto T_{011}$
## 2 (mod 8): 0
- 0:  2 -> 8   ; (18 -> 56, 26 -> 80) ; $3 \cdot T_{101} +2 -> T_{111}$
## 3 (mod 8): 0, 2
- 0:  1/3 -> 8   ; (19 -> 176, 27 -> 248) ; 9*T_001 + 5 -> T_111
- 2:  1/3 -> 2   ; (19 -> 58, 27 -> 82) ; 3*T_001 + 1 -> T_101
## 4 (mod 8): 0, 5
- 0:  4/3 -> 8   ; (20 -> 64, 28 -> 88) ; 3*T_110 + 4 -> T_111
- 5:  4/3 -> 7/3 ; (20 -> 21 , 28 -> 29) ; T_110 + 1 -> T_011
- 6 : ??
## 5 (mod 8): 0
- 0:  7/3 -> 8     ; (21 -> 64, 29 -> 88) ; 3*T_011 + 1 -> T_111
- 4:  7/3 -> 4/3   ; 
- 6 : ??
## 6 (mod 8): 0, 4, 5
- 0:  -2/9 -> 8   ; (22 -> 208, 30 -> 280) ; $9 \cdot T_{100} + 10 \mapsto T_{111}$
- 4:  -2/9 -> 4/3 ; (22 -> 68, 30 -> 92) ; $3 \cdot T_{100} + 2 \mapsto T_{110}$
- 5:  -2/9 -> 7/3 ; (22 -> 69, 30 -> 03) ; $3 \cdot T_{100} + 3 \mapsto T_{011}$
## 7 (mod 8): 0, 4, 5, 6
- 0:  -11/27 -> 8   ; (23 -> 640, 30 -> 856) ; $27 \cdot T_{000} + 19 \mapsto T_{111}$
- 4:  -11/27 -> 4/3 ; (23 -> 212, 30 -> 284) ; $9 \cdot T_{000} + 5 \mapsto T_{110}$
- 5:  -11/27 -> 7/3 ; (23 -> 213, 30 -> 285) ; $9 \cdot T_{000} + 6 \mapsto T_{011}$
- 6:  -11/27 ->-2/9 ; (23 -> , 30 -> ) ; $3 \cdot T_{000} + 3 \mapsto T_{100}$

In [144]:
20 %24, 44 % 24

(20, 20)

In [147]:
def T_010_to_T_111(k):
    return 9*k + 7
#
def T_010_to_T_110(k):
    return 3*k + 1
#
def T_010_to_T_011(k):
    return 3*k + 2
#
def T_101_to_T_111(k):
    return 3*k + 2
#
def T_001_to_T_111(k):
    return 9*k + 5
#
def T_001_to_T_101(k):
    return 3*k + 1
#
def T_110_to_T_111(k):
    return 3*k + 4
#
def T_110_to_T_011(k):
    return k + 1
#
def T_110_to_T_100(k):
    # TODO: Not correct
    if (k % 24) == 20:
        return (k-2)//3
    else:
        return k-2
#
def T_011_to_T_111(k):
    return 3*k + 1
#
def T_011_to_T_110(k):
    return k - 1
#
def T_011_to_T_100(k):
    if (k%24) == 21:
        return (k//3) -1
    else:
        return k+2
#
def T_100_to_T_111(k):
    return 9*k + 10
#
def T_100_to_T_110(k):
    return 3*k + 2
#
def T_100_to_T_011(k):
    return 3*k + 3
#
def T_000_to_T_111(k):
    return 27*k + 19
#
def T_000_to_T_110(k):
    return 9*k + 5
#
def T_000_to_T_011(k):
    return 9*k + 6
#
def T_000_to_T_100(k):
    return 3*k + 1
#


def checkFormula(mod_8, tranform_func, prefix):
    for i in range(1000):
        k = 8*(i+2) + mod_8
        label = collatzPath(k)
        label_ = prefix + label[len(prefix):]
        val_ = mrTupValue(mrTupFromPath(label_))
        if val_[1] != 1:
            print(f"Label substitution FAILED {k}({label}) -> ({label_}) has noninteger {val_}")
            return False
        k_ = tranform_func(k) 
        if k_ != val_[0]:
            print(f"{tranform_func.__name__} FAILED {k} should -> {val_} but function gave {k_}")
            return False
    return True
#
       

In [150]:
checkFormula(1, T_010_to_T_111, "111")
checkFormula(1, T_010_to_T_110, "110")
checkFormula(1, T_010_to_T_011, "011")

checkFormula(2, T_101_to_T_111, "111")
checkFormula(3, T_001_to_T_111, "111")

checkFormula(3, T_001_to_T_101, "101")

checkFormula(4, T_110_to_T_111, "111")
checkFormula(4, T_110_to_T_011, "011")
#checkFormula(4, T_110_to_T_100, "100")  removed 4(mod 8) -> 6(mod 8) case

checkFormula(5, T_011_to_T_111, "111")
checkFormula(5, T_011_to_T_110, "110")
# checkFormula(5, T_011_to_T_100, "100") removed 5(mod 8) -> 6(mod 8) case

checkFormula(6, T_100_to_T_111, "111")
checkFormula(6, T_100_to_T_110, "110")
checkFormula(6, T_100_to_T_011, "011")

checkFormula(7, T_000_to_T_111, "111")
checkFormula(7, T_000_to_T_110, "110")
checkFormula(7, T_000_to_T_011, "011")
checkFormula(7, T_000_to_T_100, "100")

True

In [ ]:
collatzPath(17)

In [ ]:
prefix = "100"
mod_8 = 7
mrTupValue(mrTupFromPath(changePrefix(collatzPath(16 + mod_8), 3, prefix))), \
    mrTupValue(mrTupFromPath(changePrefix(collatzPath(24 + mod_8), 3, prefix))), \
    mrTupValue(mrTupFromPath(changePrefix(collatzPath(32 + mod_8), 3, prefix))), \
    mrTupValue(mrTupFromPath(changePrefix(collatzPath(40 + mod_8), 3, prefix)))


# Integer Generation Rate per Lattice Segment

We know that in the bottom of the Lattice it is harder to generate integers.  

The hypothesis is that each segment of the lattice at generation a' will generate integers at a rate similar to another segment in generation a.  In other words, the slower generators just lag.


In [ ]:
def gen_rates_func(a):
    gen_rates = {}
    for a in range(4, a, 1):
        gen_rates[a] = {}
        for p in PREFIXES:
            gen_rates[a][p] = 0
        for label in generationLabels(a):
            val = mrTupValue(mrTupFromPath(label))
            if val[1] == 1:
                gen_rates[a][label[0:3]] += 1
    return gen_rates    
#
gen_rates = gen_rates_func(17)

# This is slow, pastes below
#gen_rates = gen_rates_func(27)

gen_rates_comment="""
{4: {'111': 1,
  '010': 1,
  '101': 0,
  '001': 0,
  '110': 1,
  '011': 1,
  '100': 0,
  '000': 0},
 5: {'111': 2,
  '010': 0,
  '101': 2,
  '001': 1,
  '110': 0,
  '011': 0,
  '100': 0,
  '000': 0},
 6: {'111': 2,
  '010': 1,
  '101': 0,
  '001': 0,
  '110': 2,
  '011': 2,
  '100': 1,
  '000': 0},
 7: {'111': 4,
  '010': 0,
  '101': 3,
  '001': 1,
  '110': 1,
  '011': 1,
  '100': 0,
  '000': 0},
 8: {'111': 5,
  '010': 1,
  '101': 1,
  '001': 0,
  '110': 3,
  '011': 3,
  '100': 1,
  '000': 0},
 9: {'111': 8,
  '010': 1,
  '101': 4,
  '001': 1,
  '110': 2,
  '011': 2,
  '100': 0,
  '000': 0},
 10: {'111': 10,
  '010': 2,
  '101': 3,
  '001': 2,
  '110': 4,
  '011': 4,
  '100': 1,
  '000': 0},
 11: {'111': 14,
  '010': 1,
  '101': 6,
  '001': 3,
  '110': 4,
  '011': 4,
  '100': 2,
  '000': 2},
 12: {'111': 18,
  '010': 2,
  '101': 5,
  '001': 2,
  '110': 8,
  '011': 8,
  '100': 5,
  '000': 2},
 13: {'111': 26,
  '010': 2,
  '101': 10,
  '001': 3,
  '110': 10,
  '011': 10,
  '100': 4,
  '000': 2},
 14: {'111': 36,
  '010': 3,
  '101': 12,
  '001': 3,
  '110': 14,
  '011': 14,
  '100': 5,
  '000': 2},
 15: {'111': 50,
  '010': 5,
  '101': 17,
  '001': 4,
  '110': 17,
  '011': 17,
  '100': 5,
  '000': 2},
 16: {'111': 67,
  '010': 9,
  '101': 22,
  '001': 7,
  '110': 22,
  '011': 22,
  '100': 6,
  '000': 2},
 17: {'111': 89,
  '010': 9,
  '101': 31,
  '001': 11,
  '110': 28,
  '011': 28,
  '100': 9,
  '000': 3},
 18: {'111': 117,
  '010': 14,
  '101': 37,
  '001': 12,
  '110': 40,
  '011': 40,
  '100': 14,
  '000': 3},
 19: {'111': 157,
  '010': 18,
  '101': 54,
  '001': 17,
  '110': 51,
  '011': 51,
  '100': 15,
  '000': 4},
 20: {'111': 208,
  '010': 23,
  '101': 69,
  '001': 25,
  '110': 69,
  '011': 69,
  '100': 21,
  '000': 4},
 21: {'111': 277,
  '010': 34,
  '101': 92,
  '001': 28,
  '110': 90,
  '011': 90,
  '100': 29,
  '000': 9},
 22: {'111': 367,
  '010': 44,
  '101': 124,
  '001': 47,
  '110': 121,
  '011': 121,
  '100': 37,
  '000': 8},
 23: {'111': 488,
  '010': 53,
  '101': 165,
  '001': 53,
  '110': 161,
  '011': 161,
  '100': 55,
  '000': 18},
 24: {'111': 649,
  '010': 74,
  '101': 214,
  '001': 71,
  '110': 220,
  '011': 220,
  '100': 71,
  '000': 15},
 25: {'111': 869,
  '010': 100,
  '101': 294,
  '001': 93,
  '110': 285,
  '011': 285,
  '100': 86,
  '000': 27},
 26: {'111': 1154,
  '010': 135,
  '101': 385,
  '001': 134,
  '110': 380,
  '011': 380,
  '100': 120,
  '000': 33}}
  """

In [ ]:
for p in PREFIXES:
    production = []
    for a in gen_rates:
        production.append(str(gen_rates[a][p]))
    print("\t".join(production))

# What does 0(mod 8) map to when we simply remove leading ones?

If we have a number 111..., then what other numbers 11..., 1..., ... do we HAVE to HAVE?


In [ ]:
print("\t".join(PREFIXES))

In [ ]:
1706 * 2 - 3413

In [ ]:
mrTupValue(mrTupFromPath("011"))

In [ ]:
mrTupValue(mrTupFromPath("110"))

In [ ]:
for i in range(12, 2000, 8):
    label_4 = collatzPath(i)
    if label_4[0:3] != "110":
        print(i, label_4)

In [ ]:
for a in range(4,16,1):
    """ Count the number of 7 (mod 8) numbers found per generation.
    """
    accum = [0]*8
    int_count = 0
    for label in generationLabels(a):
        T = mrTupFromPath(label)
        val = mrTupValue(T)
        if val[1] == 1:
            int_count += 1
            accum[(val[0] % 8)] += 1
            if (val[0] % 8) == 4:
                print((val[0] % 8, a, val, label, T))
            if (val[0] % 8) == 5:
                print((val[0] % 8, a, val, label, T))
    print((a, accum, int_count))
#


In [ ]:
val = mrTupValue(mrTupFromPath("0000010100100010000101100010010000001100001110101011101100011110"))
(val, val[0]/val[1])

In [ ]:

val = mrTupValue(mrTupFromPath("011010"))
(val, val[0]/val[1])

# Open Questions:

## What is _"Large Enough Generation"_ $a$ for projecting from higher density to lower density of lattice

## How would we develop 1:1 mappings from, say "111" to "000" labels?

We have the mapping that always works in the opposite direction:
```
def T_000_to_T_111(k):
    return 27*k + 19
#
```
What would mapping  be 1:1 for "111" to "000" ... it is not going to be a simple swapping of the prefix and is going to involve an arranging of the "01" extension suffix too.

So we will have something like:

"111" + tail $\mapsto$ "111"+ tail + "01"\*(generation_pad) $\mapsto$ "000" + tail + $F_?$("01"\*(generation\_pad))

to get the "000" transform for a "111" lattice node ... right?

In [ ]:
for i in range(10):
    n = 8*i + 7
    n_ = 27*n + 19
    print((n, n_), len(collatzPath(n)), len(collatzPath(n_)))

In [ ]:
for i in range(10):
    n = 8*i + 8
    n_ = n+7
    print(( n, n_ , len(collatzPath(n_)) - len(collatzPath(n)) ))

So we find we need to add a "01"*31 sized padding string pretty quickly?

In [154]:
## The 111 integers map to 101, 110, 011 prefixes 1/3 of the time

# Since 7 (mod 8) is the slowest grower, can we always convert it to everything else? YES!!!
PREFIXES =  ["111", "010", "101", "001", "110", "011", "100", "000"]
MOD =       [  0,     1,     2,     3,     4,     5,     6,     7  ]
for j in range(8):
    k = MOD[j]
    status =[[0]*8, [0]*8]
    for i in range(k+16, 10000, 8):
        label = collatzPath(i)
        for prefix in range(8):
            label_ = PREFIXES[prefix] + label[3:]
            val_ = mrTupValue(mrTupFromPath(label_))
            if val_[1] == 1:
                status[1][prefix] += 1
            else:
                status[0][prefix] += 1
    print((k, PREFIXES[j], status))


(0, '111', [[0, 1109, 832, 1109, 832, 832, 1110, 1202], [1248, 139, 416, 139, 416, 416, 138, 46]])
(1, '010', [[0, 0, 1248, 1248, 0, 0, 1248, 1248], [1248, 1248, 0, 0, 1248, 1248, 0, 0]])
(2, '101', [[0, 1248, 0, 832, 1248, 1248, 1248, 1248], [1248, 0, 1248, 416, 0, 0, 0, 0]])
(3, '001', [[0, 1248, 0, 0, 1248, 1248, 1248, 1248], [1248, 0, 1248, 1248, 0, 0, 0, 0]])
(4, '110', [[0, 832, 1248, 1248, 0, 0, 832, 1110], [1248, 416, 0, 0, 1248, 1248, 416, 138]])
(5, '011', [[0, 832, 1248, 1248, 0, 0, 832, 1110], [1248, 416, 0, 0, 1248, 1248, 416, 138]])
(6, '100', [[0, 1248, 1248, 1248, 0, 0, 0, 832], [1248, 0, 0, 0, 1248, 1248, 1248, 416]])
(7, '000', [[0, 1248, 1248, 1248, 0, 0, 0, 0], [1248, 0, 0, 0, 1248, 1248, 1248, 1248]])


In [156]:
for body in generationLabels(9):
    label = "111" + body
    T = mrTupFromPath(label)
    val = mrTupValue(T)
    if val[1] == 1:
        label_011 = "011" + label[3:]
        val_011 = mrTupValue(mrTupFromPath(label_011))
        if val_011[1] == 1:
            print(f"{val[0]}({label}) -> {val_011[0]}({label_011})")
        label_101 = "101" + label[3:]
        val_101 = mrTupValue(mrTupFromPath(label_101))
        if val_101[1] == 1:
            print(f"{val[0]}({label}) -> {val_101[0]}({label_101})")
        label_110 = "110" + label[3:]
        val_110 = mrTupValue(mrTupFromPath(label_110))
        if val_110[1] == 1:
            print(f"{val[0]}({label}) -> {val_110[0]}({label_110})")
        

4096(111111111111) -> 1365(011111111111)
4096(111111111111) -> 1364(110111111111)
1024(111111111101) -> 341(011111111101)
1024(111111111101) -> 340(110111111101)
1280(111111110111) -> 426(101111110111)
256(111111110101) -> 85(011111110101)
256(111111110101) -> 84(110111110101)
320(111111011101) -> 106(101111011101)
64(111111010101) -> 21(011111010101)
64(111111010101) -> 20(110111010101)
416(111110110111) -> 138(101110110111)
1360(111101111111) -> 453(011101111111)
1360(111101111111) -> 452(110101111111)
80(111101110101) -> 26(101101110101)
16(111101010101) -> 5(011101010101)
16(111101010101) -> 4(110101010101)
424(111011110111) -> 141(011011110111)
424(111011110111) -> 140(110011110111)
104(111011011101) -> 34(101011011101)
136(111010110111) -> 45(011010110111)
136(111010110111) -> 44(110010110111)


In [164]:
# 1280, 320, 416, 80, 104
# 1280 = 4*320 = 16 * 80
# 416 = 4 * 104
#
# 80 = 16 * 5
# 104 = 8 * 13
#
# 13 - 5 = 8

[ n//8 % 4 for n in[1280, 416, 320, 104, 80]]


[0, 0, 0, 1, 2]